#  import package

In [117]:
import pandas.io.sql as psql # pandas read sql
import pandas as pd
# import mitosheet
import time
import glob
import re
import numpy as np
import requests
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import keras
import warnings
import psycopg2
from sqlalchemy import create_engine 
import pandas.io.sql as psql # pandas read sql
import time
import os
import numpy as np
import jieba
import logging
import joblib
from tensorflow.python.keras.preprocessing import sequence,text

os.chdir('/data/jyu29/nlp/ai_forecast/supply forecast')

print('import package compelete')

try:
    # Connect to an existing database
    connection = psycopg2.connect(database="dw", user="jyu29", password="bS!3ajc#VOcLbvAR0y", host="10.50.8.227", port="60906")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result–
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    
def warn(*args, **kwargs):
    pass
warnings.warn = warn

import package compelete
PostgreSQL server information
{'user': 'jyu29', 'dbname': 'dw', 'host': '10.50.8.227', 'port': '60906', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.11 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44), 64-bit',) 



## utlis

In [115]:
def load_stopword():
#     jieba.load_userdict(r"/data/jyu29/nlp/text classfication/ilsten/data/stopwordmaster.txt")
    jieba.load_userdict(r"/data/jyu29/nlp/text classfication/ilsten/data/dictionary/joe_remark_dict.txt")

    with open(r'/data/jyu29/nlp/text classfication/ilsten/data/stopword.txt',encoding='utf8') as f:
        line_list=f.readlines()
        stopword_list=[k.strip() for k in line_list]
        stopword_set=set(stopword_list)
#         print('停顿词列表，即变量stopword_list中共有%d个元素' %len(stopword_list))
#         print('停顿词集合，即变量stopword_set中共有%d个元素' %len(stopword_set))
    return stopword_set

# remove blanks
def blank_rm(blist:list):
    """去除list中的空格项/remove blank items in the list"""
    newlist=[]
    for item in blist:
       if (item != '') and (item!=' '):
           newlist.append(item)
    
    return newlist

# wordcut and clean text
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line



# load tokenizer and transform data into sequence data
def convert_text_to_sequence(input_text):
    token = joblib.load('/data/jyu29/nlp/text classfication/dataFile_0922.pkl')
    text_sequence = token.texts_to_sequences(input_text)
    pad_text_sequence = sequence.pad_sequences(text_sequence)
    
#     sequence_text = tokenized_text.texts_to_sequences(tokenized_text)
    return pad_text_sequence

def predict_binary_class(yhat):
    yhat[yhat<=0.5]=0
    yhat[yhat>0.5]=1
    return yhat



# load data

## from db

In [175]:
check = psql.read_sql('select * from smartdata_pro.f_invalide_customer_remark order by seq_id desc limit 20;', connection) 
### change dtypes
# check[['seq_id','invalid_remark','customer_remark','flag']].seq_id.max()
check[['seq_id','invalid_remark','customer_remark','flag','order_bn']]

,seq_id,invalid_remark,customer_remark,flag,order_bn
0,546963,NaN,2058081544620964104,0,202111031410000336
1,546962,0.0,已反馈 补偿30元【迪小帆 2021.11.03 14:34:02】,0,229213250430
2,546961,1.0,自行车钢丝松动，已反馈补偿40元【迪小山 2021.11.03 14:28:48】瑕疵问题补...,0,231316853837
3,546960,0.0,请不要放驿站，否则直接差评,0,2223723204093034273
4,546959,1.0,新款,0,2074206343047278243
5,546958,1.0,2065424285728029660,0,202111031410001057
6,546957,1.0,1782254376897433246,0,202111031410002050
7,546956,1.0,2054960128785233114,0,202111031410007129
8,546955,1.0,1425537195404628092,0,202111031410005639
9,546954,1.0,2216999953573520824,0,202111031410001873


In [133]:
# check

# ad-hoc analysis

In [9]:
deli_list=['邮政',
 '韵达',
 'EMS',
 '圆通',
 'ems',
 '天天',
 '顺丰',
 '申通',
 '百世',
 '中通',
 '汇通',
 '顺风',
 '德邦',
 '完结',
 '扣单']

In [152]:
check = psql.read_sql('select * from smartdata_pro.f_invalide_customer_remark;', connection) 
# check.head()
check['date'] = check['order_time'].dt.date.astype(str)
check = check.loc[check.date>='2021-10-15']
# check1.loc[(check1['customer_remark'].str.contains('中通'))&(check1.invalid_remark==1)]

In [155]:
rnnmodel = keras.models.load_model('/data/jyu29/nlp/text classfication/invalid_remark/model/rnnmodel_0922.h5')
#     print ('Model loaded')
            #query = query.reindex(columns=model_columns, fill_value=0)
stopword = load_stopword()
check1 = check.fillna('谢谢老板')
check1['clean_text'] = check1['customer_remark'].apply(remove_punctuation)
check1.head(1)

,seq_id,order_bn,customer_remark,remark_Info,invalid_remark,order_time,flag,create_time,date,clean_text
163628,534964,202110270910006912,1917139394030152019,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":2,...",1.0,2021-10-27 09:05:48,1,2021-10-27 09:06:12.704,2021-10-27,1917139394030152019


## matched case contain keyword

In [156]:
supply =['圆通','申通','顺丰','韵达','中通','百世','天天']
# supply = ['锁单','扣单']
pattern = '|'.join(supply)

check1['MatchedValues'] = check1['customer_remark'].str.contains(pattern, case=False)

In [157]:
# matched number of case contain keyowrd
check2 = check1.loc[check1.MatchedValues==True]
check2.shape[0]

2530

In [158]:
check2['cut_text'] = check2['clean_text'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopword]))

sequence_data = convert_text_to_sequence(check2['cut_text'])
prediction = rnnmodel.predict(sequence_data)
# yhat = int(predict_binary_class(prediction))
check2['predict_test'] = prediction
check2['predict_test'] = np.where(check2['predict_test']>=0.5,1,0)
check2.head()

,seq_id,order_bn,customer_remark,remark_Info,invalid_remark,order_time,flag,create_time,date,clean_text,MatchedValues,cut_text,predict_test
168064,527481,2058651893222168701,麻烦不要发韵达，谢谢。,"[{""channel"":""迪卡侬儿童天猫旗舰店"",""commentsType"":1,""uui...",0.0,2021-10-19 08:17:57,1,2021-10-19 08:18:08.489,2021-10-19,麻烦不要发韵达谢谢,True,麻烦 不要 发 韵达 谢谢,0
168152,546779,2223445032902369262,比较着急给小孩子周5用 希望帮我加急发 如果可以发顺丰 我愿意加快递费 谢谢,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",1.0,2021-11-03 12:43:09,1,2021-11-03 12:43:28.798,2021-11-03,比较着急给小孩子周5用希望帮我加急发如果可以发顺丰我愿意加快递费谢谢,True,比较 着急 小孩子 周 5 希望 帮 加急 发 发 顺丰 愿意 加 快递费 谢谢,0
178078,535183,202110271410006293,10-27号张仕 团购订单 顺丰到付 勿动,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":2,...",1.0,2021-10-27 14:21:05,1,2021-10-27 14:21:05.848,2021-10-27,1027号张仕团购订单顺丰到付勿动,True,1027 号 张仕 团购 订单 顺丰 到付 勿动,0
184875,535221,2209895750455724781,不要发中通，收不到,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",0.0,2021-10-27 15:00:08,1,2021-10-27 15:00:17.252,2021-10-27,不要发中通收不到,True,不要 发 中通 收不到,0
189432,527732,2051277400271029505,麻烦不要发中通快递,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",0.0,2021-10-19 14:01:40,1,2021-10-19 14:01:58.961,2021-10-19,麻烦不要发中通快递,True,麻烦 不要 发 中通 快递,0


In [150]:
check2.loc[check2.predict_test==0].shape[0]

17373

In [25]:
# def flag_keyword(row):
#     if row.customer_remark.str.contains('圆通'):
#         return '圆通'
# #     elif row['customer_remark'].str.contains('申通'):
# #         return '申通'
# #     elif row['customer_remark'].str.contains('韵达'):
# #         return '韵达'
# #     elif row['customer_remark'].str.contains('天天'):
# #         return '天天'
# #     elif row['customer_remark'].str.contains('百世'):
# #         return '百世'
# #     elif row['customer_remark'].str.contains('顺丰'):
# #         return '顺丰'
# #     elif row['customer_remark'].str.contains('中通'):
# #         return '中通'
                        
#     else: 
#         return 'discard'

# #         return -1
# check1['score_ratio'] = check1.apply(flag_keyword, axis = 1)


In [73]:
# check1['flag']=check1.customer_remark.str.contains('申通')
# check1['flag']=check1.customer_remark.str.contains('中通')
# check1['flag']=check1.customer_remark.str.contains('顺丰')
# check1['flag']=check1.customer_remark.str.contains('韵达')
# check1['flag']=check1.customer_remark.str.contains('天天')
# check1['flag']=check1.customer_remark.str.contains('百世')
# check1['flag']=check1.customer_remark.str.contains('汇通')
# check1['flag']=check1.customer_remark.str.contains('申通')




In [74]:
check2 = check1.loc[check1.flag==True]

In [75]:
# add tag

# check2 = check1[check1.customer_remark.str.contains('圆通')]
check2['cut_text'] = check2['clean_text'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopword]))
check2.head(2)                   

,seq_id,order_bn,customer_remark,remark_Info,invalid_remark,order_time,flag,create_time,date,clean_text,cut_text
236471,535005,2210116431435550141,不发申通快递！,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",0.0,2021-10-27 10:22:12,True,2021-10-27 10:22:28.893,2021-10-27,不发申通快递,不发 申通 快递
264216,535042,2209206817273915439,尽量别发申通，谢谢,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",0.0,2021-10-27 11:27:56,True,2021-10-27 11:28:00.377,2021-10-27,尽量别发申通谢谢,尽量 别发 申通 谢谢


In [159]:
check1.shape,check2.shape

((22272, 11), (2530, 13))

In [160]:
check2['cut_text'] = check2['clean_text'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopword]))

sequence_data = convert_text_to_sequence(check2['cut_text'])
prediction = rnnmodel.predict(sequence_data)
# yhat = int(predict_binary_class(prediction))
check2['predict_test'] = prediction
check2['predict_test'] = np.where(check2['predict_test']>=0.5,1,0)
check2.head()


,seq_id,order_bn,customer_remark,remark_Info,invalid_remark,order_time,flag,create_time,date,clean_text,MatchedValues,cut_text,predict_test
168064,527481,2058651893222168701,麻烦不要发韵达，谢谢。,"[{""channel"":""迪卡侬儿童天猫旗舰店"",""commentsType"":1,""uui...",0.0,2021-10-19 08:17:57,1,2021-10-19 08:18:08.489,2021-10-19,麻烦不要发韵达谢谢,True,麻烦 不要 发 韵达 谢谢,0
168152,546779,2223445032902369262,比较着急给小孩子周5用 希望帮我加急发 如果可以发顺丰 我愿意加快递费 谢谢,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",1.0,2021-11-03 12:43:09,1,2021-11-03 12:43:28.798,2021-11-03,比较着急给小孩子周5用希望帮我加急发如果可以发顺丰我愿意加快递费谢谢,True,比较 着急 小孩子 周 5 希望 帮 加急 发 发 顺丰 愿意 加 快递费 谢谢,0
178078,535183,202110271410006293,10-27号张仕 团购订单 顺丰到付 勿动,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":2,...",1.0,2021-10-27 14:21:05,1,2021-10-27 14:21:05.848,2021-10-27,1027号张仕团购订单顺丰到付勿动,True,1027 号 张仕 团购 订单 顺丰 到付 勿动,0
184875,535221,2209895750455724781,不要发中通，收不到,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",0.0,2021-10-27 15:00:08,1,2021-10-27 15:00:17.252,2021-10-27,不要发中通收不到,True,不要 发 中通 收不到,0
189432,527732,2051277400271029505,麻烦不要发中通快递,"[{""channel"":""Decathlon天猫旗舰店"",""commentsType"":1,...",0.0,2021-10-19 14:01:40,1,2021-10-19 14:01:58.961,2021-10-19,麻烦不要发中通快递,True,麻烦 不要 发 中通 快递,0


In [161]:
check2.loc[check2.invalid_remark==1].shape[0]

852

In [83]:
# data records comment contain keyword
check1['flag']=check1.customer_remark.str.contains('申通')
check2 = check1.loc[check1.flag==True].shape[0]

In [39]:
# def reconfirm_deli(wordcat):
#     nlist=len(wordcat)
#     new_list=[]
#     for i in range(nlist):
#         temp_result=0
#         for item in deli_list:
#             if item in wordcat[i]:
#                 temp_result+=1
#         if temp_result>0:
#             new_list.append(0)
#         else:
#             new_list.append(1)
            
#     return new_list

## main flow test

In [163]:
'''processing and prediction '''
# author: jyu29
# create time: 2021-08

# read latest row number since last prediction
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 
import jieba
import joblib
import re
import os
import warnings
import datetime
import keras
from tensorflow.python.keras.preprocessing import sequence,text
import time 
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
def warn(*args, **kwargs):
    pass
warnings.warn = warn
logging.getLogger('tensorflow').disabled = True



def read_max_seq():
    with open(r'/data/project/invalid_remark/max_seq_id.txt','r') as f:
        max_id=f.read()
#     print(max_id)
    return max_id

# query data from datamart
def load_channel_data_with_seq(start_id):
    conn = psycopg2.connect(database="dw", user="text_miner", password="gQ@8fwta2pFtZayxxi", host="10.50.8.227", port="60906")
    c= conn.cursor()
    sql="""select seq_id, customer_remark, "remark_Info" from smartdata_pro.f_invalide_customer_remark where seq_id>=%d;"""%start_id
    c.execute(sql)
    data=c.fetchall()
    #print(data)
    raw_remarks=[]
    for item in data:
        temp_list=list(item[0:2])
        temp_json=eval(item[2])[0]
        temp_list.extend([temp_json['commentsType'],temp_json['channel']])
        temp_tuple=tuple(temp_list)
        raw_remarks.append(temp_tuple)
    
    raw_remarks=pd.DataFrame(raw_remarks,columns=['seq_id','custom_mark','comments_Type','channel'])
        
    #raw_remark=pd.DataFrame(c.fetchall(),columns=['seq_id','custom_mark','remark_info'])
    conn.close()
    
    return raw_remarks

# load stopwords
def load_stopword():
#     jieba.load_userdict(r"/data/jyu29/nlp/text classfication/ilsten/data/stopwordmaster.txt")
    jieba.load_userdict(r"/data/jyu29/nlp/text classfication/ilsten/data/dictionary/joe_remark_dict.txt")

    with open(r'/data/jyu29/nlp/text classfication/ilsten/data/stopword.txt',encoding='utf8') as f:
        line_list=f.readlines()
        stopword_list=[k.strip() for k in line_list]
        stopword_set=set(stopword_list)
#         print('停顿词列表，即变量stopword_list中共有%d个元素' %len(stopword_list))
#         print('停顿词集合，即变量stopword_set中共有%d个元素' %len(stopword_set))
    return stopword_set

# remove blanks
def blank_rm(blist:list):
    """去除list中的空格项/remove blank items in the list"""
    newlist=[]
    for item in blist:
       if (item != '') and (item!=' '):
           newlist.append(item)
    
    return newlist

# wordcut and clean text
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line



# load tokenizer and transform data into sequence data
def convert_text_to_sequence(input_text):
    token = joblib.load('/data/jyu29/nlp/text classfication/dataFile_0922.pkl')
    text_sequence = token.texts_to_sequences(input_text)
    pad_text_sequence = sequence.pad_sequences(text_sequence)
    
#     sequence_text = tokenized_text.texts_to_sequences(tokenized_text)
    return pad_text_sequence

# update prediction to datalake
def update_table(data): 
    input_data=[]
    for ind, items in data.iterrows():
        input_data.append(tuple(items))       
        
    conn = psycopg2.connect(database="dw", user="text_miner", password="gQ@8fwta2pFtZayxxi", host="10.50.8.227", port="60906")

    c= conn.cursor()
    
    c.execute("""
    update smartdata_pro.f_invalide_customer_remark as f
    set 
    invalid_remark=s.invalid_remark
    from unnest(%s) s(seq_id integer,invalid_remark integer)
    where f.seq_id=s.seq_id
    """,(input_data,))
    
    conn.commit()
    
    c.execute("""
              update smartdata_pro.f_invalide_customer_remark
set invalid_remark=0
where customer_remark like '%黑边%'""")
    conn.commit()
    conn.close()
    print('database is updated!')
    
def save_max_seq(max_seq):
    with open(r'/data/project/invalid_remark/max_seq_id.txt','w') as f:
        f.write(max_seq)
    print('Save max seq_id in file')
    print('-------------------------------------------------------------------------')
#     print(max_seq)


#     with open(r'/data/project/invalid_remark/max_seq_id.txt','r') as f:




def predict_binary_class(yhat):
    yhat[yhat<=0.5]=0
    yhat[yhat>0.5]=1
    return yhat


def user_comment_predict(data):
    rnnmodel = keras.models.load_model('/data/jyu29/nlp/text classfication/invalid_remark/model/rnnmodel_0922.h5')
#     print ('Model loaded')
            #query = query.reindex(columns=model_columns, fill_value=0)
    stopword = load_stopword()
    data = data.fillna('谢谢老板')
    data['clean_text'] = data['custom_mark'].apply(remove_punctuation)

    data['cut_text'] = data['clean_text'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopword]))
    
    ### manual add rule ###
    data.loc[data['cut_text'].str.contains('AG'), 'cut_text'] = '谢谢 老板'
    
    print('clean_text:',data['cut_text'].values)
    sequence_data = convert_text_to_sequence(data['cut_text'])
    
    prediction = rnnmodel.predict(sequence_data)
    data['invalid_remark'] = prediction
    data['invalid_remark'] = np.where(data['invalid_remark']>=0.5,1,0)
#         yhat = int(predict_binary_class(prediction))
#         data['invalid_remark'] = yhat
#     except Exception as e:
#         data['invalid_remark'] = 1
    
    return data[['seq_id','invalid_remark']]

# def customer_service_predict(data):
#     rnnmodel = keras.models.load_model('/data/jyu29/nlp/text classfication/invalid_remark/model/rnnmodel_0922.h5')
# #     print ('Model loaded')
#             #query = query.reindex(columns=model_columns, fill_value=0)
#     stopword = load_stopword()
#     data = data.fillna('谢谢老板')
#     data['clean_text'] = data['custom_mark'].apply(remove_punctuation)

#     data['cut_text'] = data['clean_text'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopword]))
#     print('clean_text:',data['cut_text'].values)

#     sequence_data = convert_text_to_sequence(data['cut_text'])
#     try:
    
#         prediction = rnnmodel.predict(sequence_data)
#         yhat = int(predict_binary_class(prediction))
#         data['invalid_remark'] = yhat
#     except Exception as e:
#         data['invalid_remark'] = 1
#     return data[['seq_id','invalid_remark']]


def predict_all_kinds(data):
    """devide data into two groups user comment and customer service comment;
    try to predict by seperate models then merge them """
#     data_customer=data[data['comments_Type']==1]
#     data_service=data[data['comments_Type']==2]
    if len(data)==0:
        print('no user comment update,the run time is ',datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        result_user=pd.DataFrame(columns=['seq_id','invalid_remark'])
    else:
        result_user=user_comment_predict(data)
    
#     if len(data_service)==0:
#         print('No customer service update, the runtime is ',datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#         result_customer=pd.DataFrame(columns=['seq_id','invalid_remark'])
#     else:
#         result_customer=customer_service_predict(data_service)
        #print(result_customer)
    
#     result_all=result_user.append(result_customer,ignore_index=True)
    result_all = result_user
#     result_all.reset_index(drop=True,inplace=True)
    result_all.seq_id = result_all.seq_id.astype(int)
    result_all.invalid_remark = result_all.invalid_remark.astype(int)

    
    return result_all
    



def predict_and_update():
    max_seq_id=read_max_seq()
#     print(max_seq_id)
    start_id=int(max_seq_id)+1
 
    try:
        query=load_channel_data_with_seq(start_id=start_id)
#         print(query)
        data_len=len(query)
    
        if data_len>0:
            final_result=predict_all_kinds(query)
            print(final_result)
            max_id=final_result['seq_id'].max()
                   
#             update_table(final_result)
#             save_max_seq(str(max_id))
            
    except Exception as e:
        print(str(e))
                
            
        
def timer(n):
    while True:
        predict_and_update()
#         print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        time.sleep(n)
    
if __name__=='__main__':
    
    warnings.filterwarnings('ignore')
    print('the process id:',os.getpid())
    timer(10)
    
    
        
        
    

the process id: 17336
clean_text: ['缺货 无补 谢春燕 111 电联 顾客 考虑 更换 商品 孙文婷 112 缺货 无补 赔付 338 元 赔付 后 客户 申请 退款 目前 未 赔付 张庆菊 113 已直 赔 缺货 补偿 338 元迪小篮 20211103140217']
   seq_id  invalid_remark
0  546879               1
clean_text: ['小 礼物']
   seq_id  invalid_remark
0  546880               1
clean_text: ['缺货 无补 谢春燕 111 电联 未 接通 已 留言 孙文婷 112 缺货 无补 客户 申请 退款 补偿 商品 金额 30 暂未 赔付 张庆菊 113'
 '24531491 缺货 无补 电联 顾客 同意 退款 迪小瑜 20211102153436 电联 顾客 愿意 退款 贵 万全 113']
   seq_id  invalid_remark
0  546881               1
1  546882               1
clean_text: ['DG2572623 缺货 无补 已转 YJ 贵 万全 113']
   seq_id  invalid_remark
0  546883               1
clean_text: ['KWHQC8524141 王申毅' 'KWHQC8524141 王申毅']
   seq_id  invalid_remark
0  546884               1
1  546885               1
clean_text: ['区域 零售 蜂鸟']
   seq_id  invalid_remark
0  546886               1
clean_text: ['发 顺丰 快递' '区域 零售 蜂鸟']
   seq_id  invalid_remark
0  546887               0
1  546888               1
clean_text: ['请 尽快 发货 谢谢']
   seq_id  invalid_remark
0  54688

KeyboardInterrupt: 